<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/data/GenerateToadCityFirewall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import random
from datetime import datetime, timedelta
import ipaddress

# Characters as users/admins
USERS = [
    'mario.bros', 'luigi.bros', 'princess.peach', 'toad', 'yoshi',
    'bowser', 'wario', 'waluigi', 'daisy', 'rosalina'
]

ADMINS = ['mario.bros', 'luigi.bros', 'princess.peach']

# Network zones
ZONES = {
    'MEDICAL': ['192.168.10.0/24', '10.10.10.0/24'],
    'ADMIN': ['192.168.20.0/24', '10.20.20.0/24'],
    'PUBLIC': ['192.168.30.0/24', '10.30.30.0/24']
}

# Common ports and services
SERVICES = {
    80: 'HTTP',
    443: 'HTTPS',
    22: 'SSH',
    3389: 'RDP',
    1433: 'SQL',
    8080: 'HTTP-ALT'
}

def generate_ip(network):
    net = ipaddress.ip_network(network)
    return str(random.choice(list(net.hosts())))

def create_database():
    conn = sqlite3.connect('firewall_logs.db')
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS firewall_logs
                 (timestamp TEXT, src_ip TEXT, dst_ip TEXT, src_port INTEGER,
                  dst_port INTEGER, protocol TEXT, action TEXT, user_id TEXT,
                  bytes_sent INTEGER, bytes_received INTEGER)''')

    return conn, c

def generate_logs(date_str, num_normal=1000, num_incidents=10):
    conn, c = create_database()
    base_date = datetime.strptime(date_str, '%Y-%m-%d')

    # Normal traffic
    for _ in range(num_normal):
        timestamp = base_date + timedelta(
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59)
        )

        src_zone = random.choice(list(ZONES.keys()))
        dst_zone = random.choice(list(ZONES.keys()))

        src_ip = generate_ip(random.choice(ZONES[src_zone]))
        dst_ip = generate_ip(random.choice(ZONES[dst_zone]))

        dst_port = random.choice(list(SERVICES.keys()))
        src_port = random.randint(49152, 65535)

        user = random.choice(USERS)

        c.execute('''INSERT INTO firewall_logs VALUES
                    (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                 (timestamp.isoformat(), src_ip, dst_ip, src_port,
                  dst_port, SERVICES[dst_port], 'ALLOW', user,
                  random.randint(1000, 100000), random.randint(1000, 100000)))

    # Generate incidents
    for _ in range(num_incidents):
        timestamp = base_date + timedelta(
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59)
        )

        incident_type = random.choice([
            'port_scan',
            'medical_zone_violation',
            'admin_access_attempt',
            'suspicious_outbound'
        ])

        if incident_type == 'port_scan':
            src_ip = generate_ip('172.16.0.0/16')  # External IP
            dst_ip = generate_ip(random.choice(ZONES['MEDICAL']))
            dst_port = random.randint(1, 65535)
            action = 'BLOCK'

        elif incident_type == 'medical_zone_violation':
            src_ip = generate_ip(random.choice(ZONES['PUBLIC']))
            dst_ip = generate_ip(random.choice(ZONES['MEDICAL']))
            dst_port = random.choice([3389, 22])
            action = 'BLOCK'

        elif incident_type == 'admin_access_attempt':
            user = random.choice([u for u in USERS if u not in ADMINS])
            src_ip = generate_ip(random.choice(ZONES['PUBLIC']))
            dst_ip = generate_ip(random.choice(ZONES['ADMIN']))
            dst_port = 3389
            action = 'BLOCK'

        else:  # suspicious_outbound
            src_ip = generate_ip(random.choice(ZONES['MEDICAL']))
            dst_ip = '185.180.143.' + str(random.randint(1, 254))
            dst_port = 443
            action = 'BLOCK'

        src_port = random.randint(49152, 65535)
        user = random.choice(USERS)

        c.execute('''INSERT INTO firewall_logs VALUES
                    (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                 (timestamp.isoformat(), src_ip, dst_ip, src_port,
                  dst_port, 'UNKNOWN', action, user,
                  random.randint(100, 1000), random.randint(100, 1000)))

    conn.commit()
    conn.close()

if __name__ == '__main__':
    generate_logs('2024-11-18')

In [ ]:
# install jupysql
!pip install jupysql -q

# set connection
%reload_ext sql

# connect to sqlite
%sql sqlite:///firewall_logs.db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.1 MB/s eta 0:00:00


Connecting to 'sqlite:///firewall_logs.db'

In [ ]:
%%sql
SELECT * FROM firewall_logs LIMIT 10

Running query in 'sqlite:///firewall_logs.db'

timestamp,src_ip,dst_ip,src_port,dst_port,protocol,action,user_id,bytes_sent,bytes_received
2024-11-18T10:14:02,10.10.10.141,192.168.10.81,60428,8080,HTTP-ALT,ALLOW,daisy,78999,88072
2024-11-18T09:20:52,10.10.10.124,10.30.30.171,60493,3389,RDP,ALLOW,rosalina,33389,54454
2024-11-18T16:11:04,192.168.30.71,192.168.30.89,62378,22,SSH,ALLOW,waluigi,53991,62086
2024-11-18T17:27:28,10.20.20.172,10.10.10.156,50296,1433,SQL,ALLOW,daisy,25025,76655
2024-11-18T00:51:47,10.30.30.218,10.30.30.175,52816,1433,SQL,ALLOW,toad,19161,95052
2024-11-18T07:17:01,10.30.30.158,192.168.20.191,52097,443,HTTPS,ALLOW,wario,52009,67824
2024-11-18T04:49:58,10.20.20.248,10.20.20.36,62626,3389,RDP,ALLOW,bowser,90250,60536
2024-11-18T00:23:15,192.168.10.150,10.10.10.195,64034,22,SSH,ALLOW,rosalina,63092,91303
2024-11-18T21:02:58,192.168.30.228,192.168.30.111,52388,80,HTTP,ALLOW,princess.peach,55503,29745
2024-11-18T01:05:55,192.168.30.140,10.10.10.194,64547,80,HTTP,ALLOW,luigi.bros,65437,85626
